In [1]:
import numpy as np
import math
import random
import os
import pybullet as p
import pybullet_data
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import matplotlib.pyplot as plt
import time

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory

In [4]:
class BikeEnv(gym.Env):
    def __init__(self):
        p.connect(p.GUI)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(np.array([-1000]*10), np.array([1000]*10))
        self.timestep = 1./240.
        
    def step(self, action):
        #print("Action:",action)
        
        
        if action==0 :  
            p.setJointMotorControl2(self.pid, 1, p.VELOCITY_CONTROL, targetVelocity=0, force=0)  
        else:  
            p.setJointMotorControl2(self.pid, 1, p.VELOCITY_CONTROL, targetVelocity=10, force=10)
         
       # if action[1] < 0.5 :   
       #     p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=0, force=0) 
       # else:  
       #     p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=5, force=5)    
    
        p.stepSimulation()
        #time.sleep(self.timestep)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.7 or  state[2] >= 2:
            reward = -100
            done = True
        else :
            reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        state_object ,orient = p.getBasePositionAndOrientation(self.pid)
        norm = math.sqrt(orient[0]**2+orient[1]**2+orient[2]**2+orient[3]**2)
        observation =velocity[0]+velocity[1]+ orient
        
        info = {'x':state_object[0],'y':state_object[1],'z':state_object[2]}
        return observation, reward, done, info
            
        
    def reset(self):
        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        #p.setRealTimeSimulation(0)
        p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=0, force=0) 
        
        velocity = p.getBaseVelocity(self.pid)
        state_object ,orient = p.getBasePositionAndOrientation(self.pid)
        norm = math.sqrt(orient[0]**2+orient[1]**2+orient[2]**2+orient[3]**2)
        observation =velocity[0]+velocity[1]+ orient
        
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        return observation
        
    def render(self, mode='human'):
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0.7,0,0.05],
                                                            distance=.7,
                                                            yaw=90,
                                                            pitch=-70,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=960,
                                              height=720,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (720,960, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
nb_actions = env.action_space.n

In [7]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                176       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)               

In [8]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.fit(env, nb_steps=50000, visualize=True, verbose=2)

Training for 50000 steps ...



C:\Users\SERVER\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   330/50000: episode: 1, duration: 27.198s, episode steps: 330, steps per second: 12, episode reward: 229.000, mean reward: 0.694 [-100.000, 1.000], mean action: 0.482 [0.000, 1.000], mean observation: 0.147 [-2.771, 1.600], loss: 0.098475, mae: 1.046750, mean_q: 1.916174
   659/50000: episode: 2, duration: 24.909s, episode steps: 329, steps per second: 13, episode reward: 228.000, mean reward: 0.693 [-100.000, 1.000], mean action: 0.653 [0.000, 1.000], mean observation: 0.157 [-2.847, 1.538], loss: 11.189943, mae: 2.416688, mean_q: 4.481734
   989/50000: episode: 3, duration: 24.908s, episode steps: 330, steps per second: 13, episode reward: 229.000, mean reward: 0.694 [-100.000, 1.000], mean action: 0.527 [0.000, 1.000], mean observation: 0.142 [-2.773, 1.565], loss: 8.908719, mae: 3.797318, mean_q: 7.445909
  1319/50000: episode: 4, duration: 25.512s, episode steps: 330, steps per second: 13, episode reward: 229.000, mean reward: 0.694 [-100.000, 1.000], mean action: 0.533 [0.000, 